In [ ]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf
from tensorflow import keras

mnist_dataset = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

n_epochs = 20   # Number of optimization epochs
n_layers = 1    # Number of random layers
n_train = 200    # Size of the train dataset
n_test = 120     # Size of the test dataset
n_batches = 4     # Size of the batches

# Reduce dataset size
train_images = train_images[:n_train]
train_labels = train_labels[:n_train]
test_images = test_images[:n_test]
test_labels = test_labels[:n_test]

# Normalize pixel values within 0 and 1
train_images = train_images / 255
test_images = test_images / 255

# Add extra dimension for convolution channels
train_images = np.array(train_images[..., tf.newaxis], requires_grad=False)
test_images = np.array(test_images[..., tf.newaxis], requires_grad=False)

n_qubits = 4

dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface='tf')
def qnotnode(inputs):
    inputs *= np.pi
    # Encoding of 4 classical input values
    qml.AngleEmbedding(inputs, wires=range(n_qubits), rotation='Y')

    # Filter from arxiv.org/abs/2308.14930

    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[0, 3])


    # Measurement producing 4 classical output values
    return [qml.expval(qml.PauliZ(j)) for j in range(n_qubits)]

class ConvQLayer(qml.qnn.KerasLayer):
    
    def call(self, inputs):

        out = tf.Variable(tf.zeros((n_batches, 14, 14, n_qubits)))
        for b in range(n_batches):
            # Loop over the coordinates of the top-left pixel of 2X2 squares
            for j in range(0, 28, 2):
                for k in range(0, 28, 2):
                    # Process a squared 2x2 region of the image with a quantum circuit
                    q_results = tf.stack(
                        [
                            inputs[b,j, k, 0],
                            inputs[b,j, k + 1, 0],
                            inputs[b,j + 1, k, 0],
                            inputs[b,j + 1, k + 1, 0]
                        ],
                        axis = 1
                    )
                    q_results = super().call(q_results)
                    # Assign expectation values to different channels of the output pixel (j/2, k/2)
                    for c in range(n_qubits):
                        out[b,j // 2, k // 2, c] = q_results[c]
        return out 

qnotlayer = qml.qnn.KerasLayer(qnotnode, {}, output_dim=[28,28,n_qubits])

qnotlayer.trainable = False

In [ ]:
f, ax = plt.subplots(nrows=4, ncols=4, figsize=(12, 4))
for i, title in enumerate(image_titles):
    ax[0, i].set_title(title, fontsize=16)
    ax[0, i].imshow(activations[i,:,:,0])
    ax[0, i].axis('off')
    ax[1, i].imshow(activations[i,:,:,1])
    ax[1, i].axis('off')
    ax[2, i].imshow(activations[i,:,:,2])
    ax[2, i].axis('off')
    ax[3, i].imshow(activations[i,:,:,3])
    ax[3, i].axis('off')
    
plt.tight_layout()
plt.show()

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np

import matplotlib.animation as animation

fig, ax = plt.subplots()


def f(x, y):
    return np.sin(x) + np.cos(y)

x = np.linspace(0, 2 * np.pi, 120)
y = np.linspace(0, 2 * np.pi, 100).reshape(-1, 1)

# ims is a list of lists, each row is a list of artists to draw in the
# current frame; here we are just animating one artist, the image, in
# each frame
ims = []
for i in range(60):
    x += np.pi / 15
    y += np.pi / 30
    im = ax.imshow(f(x, y), animated=True)
    if i == 0:
        ax.imshow(f(x, y))  # show an initial one first
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True,
                                repeat_delay=1000)

plt.show()